<a href="https://colab.research.google.com/github/x-legion/legions-sd-notebooks/blob/main/One_Click_Oobabooga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown 🎵`Press play on the music player that will appear below:`
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 1.1 Install and Run WebUI { display-mode: "form" }
!pip install requests_file pyrfc6266

import shutil
import os, re, csv, requests, pyrfc6266
from IPython.display import clear_output
!apt -y update -qq
!apt install aria2
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

#@markdown ⚠️`Google drive is limited to 15GB storage only`
everything_on_google_drive = False #@param {type:"boolean"}
#@markdown `Insert necessary ` [Basic Settings](https://github.com/oobabooga/text-generation-webui/blob/main/README.md#basic-settings) ` in launch_arguments`
launch_arguments = "--share --chat --settings settings-colab-template.json" #@param ["--share --chat --settings settings-colab-template.json"] {allow-input: true}

#@markdown ---
#@markdown #**Download Models**
#@markdown `Insert any` [HuggingFace](https://huggingface.co/TheBloke) `model repo in <Organization>/<model> format`
hf_model_download = "" # @param ["TheBloke/Manticore-13B-Chat-Pyg-Guanaco-SuperHOT-8K-GPTQ", "TheBloke/WizardCoder-15B-1.0-GPTQ", "TheBloke/WizardLM-13B-V1-1-SuperHOT-8K-GPTQ", "TheBloke/Pygmalion-13B-SuperHOT-8K-GPTQTheBloke/Pygmalion-13B-SuperHOT-8K-GPTQ", "TheBloke/OpenAssistant-Llama2-13B-Orca-8K-3319-GPTQ", ""] {allow-input: true}
# @markdown `Insert any` [HuggingFace](https://huggingface.co/TheBloke) `model links for download. (Input multiple links separated by comma)`
download_links = "" # @param [""] {allow-input: true}


# everything on google drive
if everything_on_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive
  repo_dir = '/content/drive/MyDrive/text-generation-webui'
  if os.path.exists(repo_dir):
    %cd {repo_dir}
    !git pull
  else:
    !git clone https://github.com/oobabooga/text-generation-webui.git

else:
  %cd /content
  repo_dir = '/content/text-generation-webui'
  if os.path.exists(repo_dir):
    %cd {repo_dir}
    !git pull
  else:
    !git clone https://github.com/oobabooga/text-generation-webui.git


%cd {repo_dir}
model_dir = f"{repo_dir}/models"

# hf model download
!python download-model.py {hf_model_download}

#downloader
%cd {repo_dir}
links = list(csv.reader([download_links]))[0]
session = requests.session()
for link in links:
  try:
    response = session.get(link, stream=True)
    file_name = pyrfc6266.requests_response_to_filename(response)
    print(f"Downloading {file_name} to {model_dir}")
    !aria2c -x 16 -s 16 -d {model_dir} {link} -o {file_name}
    clear_output()
  except Exception as e:
    print(f"Error: {e}")

# install requirements
%cd {repo_dir}
!wget https://raw.githubusercontent.com/pcrii/Philo-Colab-Collection/main/settings-colab-template.json -O settings-colab-template.json
!pip install -r requirements.txt
# !pip install -r /extensions/api/requirements.txt
# !pip install -r /extensions/google_translate/requirements.txt
# !pip install -r /extensions/ngrok/requirements.txt
# !pip install -r /extensions/elevenlabs_tts/requirements.txt
# !pip install -r /extensions/openai/requirements.txt
# !pip install -r /extensions/silero_tts/requirements.txt
# !pip install -r /extensions/superbooga/requirements.txt
# !pip install -r /extensions/whisper_stt/requirements.txt

!pip uninstall -y llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir
clear_output(wait=True)

# list models
print("Models Directory")
print(os.listdir(model_dir))

# run webui
!python server.py {launch_arguments}

In [ ]:
#@title # Run WebUI `in case of OOM` { display-mode: "form" }

!python server.py {launch_arguments}
